In [1]:
import uhd
import numpy as np
usrp = uhd.usrp.MultiUSRP(args='')

[INFO] [UHD] linux; GNU C++ version 9.4.0; Boost_107100; UHD_3.15.0.HEAD-0-gaea0e2de


RuntimeError: LookupError: KeyError: No devices found for ----->
Empty Device Address

In [ ]:
samples = 0.1*np.random.randn(10000) + 0.1j*np.random.randn(10000) # create random signal
duration = 10 # seconds
center_freq = 915e6
sample_rate = 1e6
gain = 20 # [dB] start low then work your way up
usrp.send_waveform(samples, duration, center_freq, sample_rate, [0], gain)

[INFO] [UHD] linux; GNU C++ version 9.4.0; Boost_107100; UHD_3.15.0.HEAD-0-gaea0e2de
[INFO] [B200] Loading firmware image: /usr/local/share/uhd/images/usrp_b200_fw.hex...
[INFO] [B200] Detected Device: B210
[INFO] [B200] Loading FPGA image: /usr/local/share/uhd/images/usrp_b210_fpga.bin...
[INFO] [B200] Operating over USB 3.
[INFO] [B200] Detecting internal GPSDO.... 
[INFO] [GPS] No GPSDO found
[INFO] [B200] Initialize CODEC control...
[INFO] [B200] Initialize Radio control...
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Setting master clock rate selection to 'automatic'.
[INFO] [B200] Asking for clock rate 16.000000 MHz... 
[INFO] [B200] Actually got clock rate 16.000000 MHz.
[INFO] [B200] Asking for clock rate 56.000000 MHz... 
[INFO] [B200] Actually got clock rate 56.000000 MHz.
